# Librerias

In [ ]:
%%capture 
!pip install transformers

In [ ]:
from sklearn.model_selection import train_test_split
import torch

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%capture
!pip install wandb
import wandb
wandb.login()

wandb: Paste an API key from your profile and hit enter: ··········


In [ ]:
wandb.login()

wandb: Currently logged in as: lucialarraona (use `wandb login --relogin` to force relogin)


True

# Carga de datos
- incluidas nuevas columnas con concatenacion de review_body y categoria, review_body y titulo
- nuevos indices para star rating, de 0-4 en vez de 1-5
- numericalizar las categorias de los productos, 31 en total

Para TRAIN:
  - data ingles EN 
  - data aleman DE

Para VALID Y TEST:
  - data en ingles EN
  - data en aleman DE
  - data en español ES
  - data en francés FR
  - data en japones JA
  - data en chino ZH

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load data
import pandas as pd

## Para TRAIN, en ingles y en aleman

path1 = "/content/drive/MyDrive/TFG/Datasets_procesados_2/train/amazonEN_train.csv"
#path1 = "drive/MyDrive/TFG_1/Datasets_procesados/train/amazonDE_train.csv"

## Para VALID, en ingles, aleman, español, frances, japones y chino

#path2 = "drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonEN_valid.csv"
path2 = "/content/drive/MyDrive/TFG/Datasets_procesados_2/valid/amazonDE_valid.csv"
#path2 = "/content/drive/MyDrive/TFG/Datasets_procesados_2/valid/amazonES_valid.csv"
#path2 = "content/drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonFR_valid.csv"
#path2 = "content/drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonJA_valid.csv"
#path2 = "content/drive/MyDrive/TFG_1/Datasets_procesados/valid/amazonZH_valid.csv"

## Para TEST, en ingles, aleman, español, frances, japones y chino

#path3 = "drive/MyDrive/TFG_1/Datasets_procesados/test/amazonEN_test.csv"
path3 = "/content/drive/MyDrive/TFG/Datasets_procesados_2/test/amazonDE_test.csv"
#path3 = "/content/drive/MyDrive/TFG/Datasets_procesados_2/test/amazonES_test.csv"
#path3 = "drive/MyDrive/TFG_1/Datasets_procesados/test/amazonFR_test.csv"
#path3 = "drive/MyDrive/TFG_1/Datasets_procesados/test/amazonJA_test.csv"
#path3 = "drive/MyDrive/TFG_1/Datasets_procesados/test/amazonZH_test.csv"



####### CONVERTIMOS A DATAFRAMES #########

df_train = pd.read_csv(path1)
df_train = df_train

df_valid = pd.read_csv(path2)
df_valid = df_valid

df_test = pd.read_csv(path3)
df_test = df_test

In [ ]:
df_train.head(1)

,review_id,product_id,reviewer_id,star_rating,review_body,review_title,language,product_category,text_title,text_title_cat,text_cat,product_category_cod
0,en_0522546,product_en_0681589,reviewer_en_0687817,2,Not strong enough to run a small 120v vacuum c...,Not strong enough to run a small 120v vacuum c...,en,lawn_and_garden,Not strong enough to run a small 120v vacuum c...,Not strong enough to run a small 120v vacuum c...,Not strong enough to run a small 120v vacuum c...,17


# Carga modelo mBERT
- incluye tokenizer

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-multilingual-cased'
from transformers import BertModel, BertConfig, BertTokenizer
from transformers import BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# mBertForSequenceClassification - RATING
- Clasificación Rating a partir de review_body

In [ ]:
%env WANDB_PROJECT = RATING CLASSIFICATION mBERT 

env: WANDB_PROJECT=RATING CLASSIFICATION mBERT


In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = ['0','1','2','3','4'] 

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.star_rating.values
val_labels = df_valid.star_rating.values
test_labels = df_test.star_rating.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
from transformers import BertForSequenceClassification

## DEFINIMOS EL MODELO 

model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
#from datasets import load_metric
#import numpy as np

#accuracy_metric = load_metric("accuracy")

#def compute_metrics(eval_pred):
 #   predictions, labels = eval_pred
  #  predictions = np.argmax(predictions, axis=1)
    # metrics from the datasets library have a `compute` method
    #  return accuracy_metric.compute(predictions=predictions, references=labels)

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    load_best_model_at_end = True,
    #report_to = 'wandb',           # enable logging to W&B
    run_name = 'EN - DE'   # name of the W&B run
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,           # training dataset
    eval_dataset = test_data,             # evaluation dataset
    compute_metrics=compute_metrics            
)

trainer.train()


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

## Evaluation

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.395,
 'eval_f1': 0.4023793173405573,
 'eval_loss': 1.5324207544326782,
 'eval_precision': 0.4795595150484503,
 'eval_recall': 0.395,
 'eval_runtime': 35.7113,
 'eval_samples_per_second': 28.002}

##Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

Rating_text_mBERT


In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_mBERT"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_mBERT/tokenizer_config.json',
 'drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_mBERT/special_tokens_map.json',
 'drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_mBERT/vocab.txt',
 'drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_mBERT/added_tokens.json')

# mBertForSequenceClassification - RATING
- Clasificación Rating a partir de review_body + titulo

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = ['0','1','2','3','4'] 

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.star_rating.values
val_labels = df_valid.star_rating.values
test_labels = df_test.star_rating.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
## Lo definimos de nuevo unicamente con el proposito de usar las secciones del documento de forma independiente 

model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,         # training dataset
    eval_dataset = test_data,
    compute_metrics=compute_metrics       # evaluation dataset             
)

trainer.train()

## Evaluation

In [ ]:
trainer.evaluate()

## Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_title_mBERT"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# mBertForSequenceClassification - RATING
- Clasificación Rating a partir de review_body + titulo + categoria

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = ['0','1','2','3','4'] 
#class_names = ['0','1'] 
#class_names = ['1']

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.text_title_cat.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.text_title_cat.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.text_title_cat.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.star_rating.values
val_labels = df_valid.star_rating.values
test_labels = df_test.star_rating.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,         # training dataset
    eval_dataset = test_data,             # evaluation dataset
    compute_metrics=compute_metrics             
)

trainer.train()

## Evaluation

In [ ]:
trainer.evaluate()

## Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG_1/Modelos_entrenados/Rating_text_title_cat_mBERT"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# BertForSequenceClassification - CATEGORÍA

- Clasificación de categorias de productos a partir del review_body

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = df_train.product_category.unique().tolist()

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.review_body.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.product_category_cod.values
val_labels = df_valid.product_category_cod.values
test_labels = df_test.product_category_cod.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,         # training dataset
    eval_dataset = test_data,            # evaluation dataset
    compute_metrics = compute_metrics             
)

trainer.train()

## Evaluation

In [ ]:
trainer.evaluate()

## Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG_1/Modelos_entrenados/Category_text_mBERT"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

# BertForSequenceClassification - CATEGORÍA

- Clasificación de categorias de productos a partir del review_body + titulo

In [ ]:
RANDOM_SEED=8
MAX_LENGTH=512


## CLASES DEL CLASIFICADOR 

class_names = df_train.product_category.unique().tolist()

In [ ]:
#TOKENIZAMOS LOS DATAFRAMES

train_encodings = tokenizer(df_train.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

val_encodings = tokenizer(df_valid.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

test_encodings = tokenizer(df_test.text_title.values.tolist(), #truncation=True, padding=True,
      add_special_tokens=True,
      truncation=True,
      max_length=MAX_LENGTH,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')

In [ ]:
class GPReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_labels = df_train.product_category_cod.values
val_labels = df_valid.product_category_cod.values
test_labels = df_test.product_category_cod.values

In [ ]:
train_data = GPReviewDataset(train_encodings, torch.from_numpy(train_labels))
val_data = GPReviewDataset(val_encodings, torch.from_numpy(val_labels))
test_data = GPReviewDataset(test_encodings, torch.from_numpy(test_labels))

In [ ]:
model = BertForSequenceClassification.from_pretrained(
    PRE_TRAINED_MODEL_NAME, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = len(class_names), # The number of output labels--2 for binary classification.  si pones num_labels=1 hace MSE LOSS
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False ,# Whether the model returns all hidden-states.   
    vocab_size=tokenizer.vocab_size)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## Training

In [ ]:
model_dir="./"

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = model_dir +  'prueba/results',          # output directory
    overwrite_output_dir = True,
    evaluation_strategy='epoch',
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir = model_dir +  'prueba/logs',            # directory for storing logs
    logging_steps=10,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model = model,                         # the instantiated 🤗 Transformers model to be trained
    args = training_args,                  # training arguments, defined above
    train_dataset = train_data,         # training dataset
    eval_dataset = test_data,
    compute_metrics = compute_metrics            # evaluation dataset
)

trainer.train()

## Evaluation

In [ ]:
trainer.evaluate()

## Save Model

In [ ]:
!ls drive/MyDrive/TFG_1/Modelos_entrenados/

In [ ]:
# saving the fine tuned model & tokenizer
model_path = "drive/MyDrive/TFG_1/Modelos_entrenados/Category_text_title_mBERT"
trainer.save_model(model_path)
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)